In [4]:
import tensorflow as tf
import numpy as np

/Users/monk/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/monk/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [51]:
tf.reset_default_graph()

class lstm_Attention_with_Pos(object):
    
    
    def __init__(self,vocab_size,embedding_dim,num_cell,dropout_value,pos_vocab_size,pdim,category_size):
        
        
        
        
        sentence = tf.placeholder(name='input_data',shape=[None,None],dtype=tf.int32)
        pos_     = tf.placeholder(name='pos',shape=[None,None],dtype=tf.int32)
        intents  = tf.placeholder(name='labels',shape=[None],dtype=tf.int32)
        mode =  tf.placeholder(name='dropout',shape=(),dtype=tf.int32)
        
        
        
        
        
        self.placeholders = {
            
                            'input_d':sentence,
                            'labels':intents,
                             'pos':pos_,
                            'dropout':mode
                            }
        

        
        
        embedding_matrix = tf.get_variable(name='word_embedding',
                                           shape=[vocab_size,embedding_dim],
                                           dtype=tf.float32,
                                           initializer=tf.random_uniform_initializer(-0.01,0.01))
        
        embedd_word2vec = tf.nn.embedding_lookup(embedding_matrix,sentence)
        
        dropout = tf.cond(
                          tf.equal(mode,0),
                          lambda : dropout_value, 
                          lambda : 0.
                         )
        
        pos_embedding = tf.get_variable(name='pos_embedding',
                                           shape=[pos_vocab_size,pdim],
                                           dtype=tf.float32,
                                           initializer=tf.random_uniform_initializer(-0.01,0.01))
        
        
        pos_lookup = tf.nn.embedding_lookup(pos_embedding, pos_ )
        
        final_input = tf.concat([embedd_word2vec,pos_lookup],axis=-1)
        
        
                
        
        
        
        
        sequence_length = tf.count_nonzero(sentence,axis=-1)
        
        
        with tf.variable_scope('forward_cell'):
                lstm_cell_fw = tf.contrib.rnn.LSTMCell(num_cell,forget_bias=1.0)
                dropout_wrapper_fw = tf.contrib.rnn.DropoutWrapper(lstm_cell_fw,output_keep_prob=1. - dropout_value)
        
        
        with tf.variable_scope('backward_cell'):
            lstm_cell_bw = tf.contrib.rnn.LSTMCell(num_cell,forget_bias=1.0)
            dropout_wrapper_bw = tf.contrib.rnn.DropoutWrapper(lstm_cell_bw,output_keep_prob=1.- dropout_value)
        
        
        with tf.variable_scope('bi_lstm') as scope:
            output , last_state     = tf.nn.bidirectional_dynamic_rnn(dropout_wrapper_fw,
                                                      dropout_wrapper_bw,
                                                      final_input,
                                                      sequence_length=sequence_length,
                                                      dtype=tf.float32)
        
        logits = tf.concat(output,2)
        #ex . 12 x 10 x 24
        
        
        
        
        #Attention_layer
        
        
        #ex : 12x10x24 . ===> 120 x 24
        input_reshape = tf.reshape(logits,[-1,num_cell*2])
        
        
        #num_cell = 12
        #ex : # 24 x 1
        attention_size = tf.get_variable(name='attention_size',
                                         shape=[2*num_cell,1],
                                         dtype=tf.float32,
                                         initializer=tf.random_uniform_initializer(-0.01,0.01))
        # bias 1
        bias          = tf.get_variable(name='bias',shape=[1],
                                        dtype=tf.float32,
                                        initializer=tf.random_uniform_initializer(-0.01,0.01))
        
        
        #projection without activation 
        #ex : 120x24 matmul 24x 1 ==> 120x1
        attention_projection = tf.add(tf.matmul(input_reshape,attention_size),bias)
        
        
        #reshape . 120x1 ==> 12x10x1 (shape of input )
        output_reshape = tf.reshape(attention_projection,[tf.shape(sentence)[0],tf.shape(sentence)[1],-1])
        
        #softmax over logits 12x10x1
        attention_output = tf.nn.softmax(output_reshape,dim=1)
        
        
        #reshape as input 12x10
        attention_visualize = tf.reshape(attention_output,
                                         [tf.shape(sentence)[0],
                                          tf.shape(sentence)[1]],
                                         name='Plot')
        
        
        # 12x10x1 multiply 12x10x24  == > 12x10x24
        attention_projection_output = tf.multiply(attention_output,logits)
        
        #reduce across time 120x10x24 ==> 12x24
        Final_output = tf.reduce_sum(attention_projection_output,1)
        
        
        
        #fully_connected_layer
        
        f_c = tf.get_variable(name='f_c_layer',
                              shape=[2*num_cell,category_size],
                              dtype=tf.float32,
                              initializer=tf.random_uniform_initializer(-0.01,0.01))
        
        
        f_bias = tf.get_variable(name='fc_bias',
                                 shape=[category_size],
                                dtype=tf.float32,
                                 initializer=tf.random_uniform_initializer(-0.01,0.01))
        
        output_projection = tf.add(tf.matmul(Final_output,f_c),f_bias)
        
        
        
        probability_dist  = tf.nn.softmax(output_projection,name='pred')
        
        prediction  = tf.argmax(probability_dist,axis=-1)
        
        
        #as_usual_stuff_
        cross_entropy  = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=intents,logits=output_projection)
        loss = tf.reduce_mean(cross_entropy)
        accuracy_calculation = tf.reduce_mean(tf.cast(tf.equal(tf.cast(prediction,tf.int32),intents),tf.float32))
        
        
        
        self.output = {  'loss':loss,
                         'accuracy':accuracy_calculation,
                         'prediction':prediction,
                         'probai':probability_dist,
                         'logits':output_projection,
                         'attention_visualize':attention_visualize
            
                       }
        
        
        # this guy is hero of this movie
        self.train = tf.train.AdamOptimizer().minimize(loss)
        

def execute_model(model):
    
    with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())
        
        for i in range(1000):
            outp,train = sess.run([model.output,model.train],feed_dict={model.placeholders['input_d']:np.random.randint(0,3,[12,10]),
                                                model.placeholders['labels']:np.random.randint(0,2,[12,]),
                                                model.placeholders['dropout']:0,
                                                model.placeholders['pos']: np.random.randint(0,3,[12,10]) })
            print(outp['loss'],outp['accuracy'],outp['prediction'].shape,outp['attention_visualize'].shape)
        
        
if __name__ == '__main__':
    
    model = lstm_Attention_with_Pos(130,100,12,0.5,120,22,12)
    execute_model(model)
    
    
    
    
        
        
        
        
        
        
        
        
        
        
        
        
        

2.4860346 0.0 (12,) (12, 10)
2.4869893 0.0 (12,) (12, 10)
2.483797 0.0 (12,) (12, 10)
2.482539 0.0 (12,) (12, 10)
2.4796963 0.5833333 (12,) (12, 10)
2.4785538 0.5 (12,) (12, 10)
2.4755871 0.5833333 (12,) (12, 10)
2.4752314 0.33333334 (12,) (12, 10)
2.472325 0.41666666 (12,) (12, 10)
2.4664872 0.6666667 (12,) (12, 10)
2.4631197 0.75 (12,) (12, 10)
2.462633 0.5833333 (12,) (12, 10)
2.4616535 0.41666666 (12,) (12, 10)
2.454745 0.5 (12,) (12, 10)
2.4530993 0.41666666 (12,) (12, 10)
2.4463744 0.5 (12,) (12, 10)
2.4463093 0.41666666 (12,) (12, 10)
2.4389744 0.41666666 (12,) (12, 10)
2.4331634 0.41666666 (12,) (12, 10)
2.4246128 0.5 (12,) (12, 10)
2.4152336 0.5833333 (12,) (12, 10)
2.4078074 0.5833333 (12,) (12, 10)
2.4023492 0.5833333 (12,) (12, 10)
2.4048345 0.41666666 (12,) (12, 10)
2.3900135 0.5 (12,) (12, 10)
2.3767135 0.6666667 (12,) (12, 10)
2.3655512 0.41666666 (12,) (12, 10)
2.3702717 0.33333334 (12,) (12, 10)
2.331257 0.5 (12,) (12, 10)
2.3070354 0.5833333 (12,) (12, 10)
2.316922 0.

0.6949628 0.5833333 (12,) (12, 10)
0.67350847 0.75 (12,) (12, 10)
0.7298846 0.33333334 (12,) (12, 10)
0.74640393 0.25 (12,) (12, 10)
0.7191043 0.41666666 (12,) (12, 10)
0.6950829 0.5833333 (12,) (12, 10)
0.7084492 0.5 (12,) (12, 10)
0.69574237 0.5833333 (12,) (12, 10)
0.6879735 0.6666667 (12,) (12, 10)
0.70050305 0.5833333 (12,) (12, 10)
0.7220877 0.33333334 (12,) (12, 10)
0.7262743 0.33333334 (12,) (12, 10)
0.6767745 0.8333333 (12,) (12, 10)
0.7038781 0.5 (12,) (12, 10)
0.68485135 0.6666667 (12,) (12, 10)
0.6955417 0.6666667 (12,) (12, 10)
0.69673723 0.5833333 (12,) (12, 10)
0.6894481 0.6666667 (12,) (12, 10)
0.7135331 0.41666666 (12,) (12, 10)
0.69568187 0.5833333 (12,) (12, 10)
0.73190325 0.33333334 (12,) (12, 10)
0.6604676 0.75 (12,) (12, 10)
0.71064186 0.5 (12,) (12, 10)
0.69455296 0.5833333 (12,) (12, 10)
0.72296864 0.41666666 (12,) (12, 10)
0.7483193 0.25 (12,) (12, 10)
0.72221136 0.41666666 (12,) (12, 10)
0.70617986 0.5 (12,) (12, 10)
0.7135814 0.41666666 (12,) (12, 10)
0.72906

0.69261914 0.5833333 (12,) (12, 10)
0.64198965 0.75 (12,) (12, 10)
0.68458396 0.5833333 (12,) (12, 10)
0.6929343 0.5833333 (12,) (12, 10)
0.7289333 0.41666666 (12,) (12, 10)
0.7085684 0.5 (12,) (12, 10)
0.6901701 0.5833333 (12,) (12, 10)
0.7224908 0.41666666 (12,) (12, 10)
0.7431891 0.25 (12,) (12, 10)
0.66368014 0.75 (12,) (12, 10)
0.69518155 0.5833333 (12,) (12, 10)
0.7089738 0.41666666 (12,) (12, 10)
0.69293207 0.5 (12,) (12, 10)
0.72339696 0.25 (12,) (12, 10)
0.7178877 0.16666667 (12,) (12, 10)
0.70482296 0.41666666 (12,) (12, 10)
0.6894409 0.5833333 (12,) (12, 10)
0.70418054 0.25 (12,) (12, 10)
0.70135206 0.41666666 (12,) (12, 10)
0.70758486 0.33333334 (12,) (12, 10)
0.70676595 0.41666666 (12,) (12, 10)
0.7041629 0.5 (12,) (12, 10)
0.69403553 0.5 (12,) (12, 10)
0.703878 0.41666666 (12,) (12, 10)
0.7025878 0.41666666 (12,) (12, 10)
0.6942885 0.41666666 (12,) (12, 10)
0.69825476 0.5833333 (12,) (12, 10)
0.6987179 0.6666667 (12,) (12, 10)
0.6951856 0.5833333 (12,) (12, 10)
0.6968613 

0.6864021 0.5833333 (12,) (12, 10)
0.69236976 0.5833333 (12,) (12, 10)
0.7066304 0.5 (12,) (12, 10)
0.72807384 0.33333334 (12,) (12, 10)
0.696857 0.5833333 (12,) (12, 10)
0.7013993 0.5 (12,) (12, 10)
0.72100806 0.25 (12,) (12, 10)
0.69788665 0.41666666 (12,) (12, 10)
0.694392 0.41666666 (12,) (12, 10)
0.6924403 0.6666667 (12,) (12, 10)
0.7042907 0.33333334 (12,) (12, 10)
0.71275425 0.083333336 (12,) (12, 10)
0.6944136 0.5833333 (12,) (12, 10)
0.68990237 0.5833333 (12,) (12, 10)
0.7116826 0.33333334 (12,) (12, 10)
0.701422 0.41666666 (12,) (12, 10)
0.6823124 0.6666667 (12,) (12, 10)
0.68300223 0.5833333 (12,) (12, 10)
0.6990288 0.41666666 (12,) (12, 10)
0.69932455 0.5 (12,) (12, 10)
0.69130135 0.5833333 (12,) (12, 10)
0.6848526 0.5833333 (12,) (12, 10)
0.7017563 0.33333334 (12,) (12, 10)
0.68109185 0.6666667 (12,) (12, 10)
0.6869569 0.5833333 (12,) (12, 10)
0.6797809 0.75 (12,) (12, 10)
0.6730833 0.6666667 (12,) (12, 10)
0.7485292 0.16666667 (12,) (12, 10)
0.6986032 0.5833333 (12,) (12,